Import Libraries

In [1]:
import jax
import jax.numpy as jnp
import jax.nn as jnn
from flax import nnx
import optax
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import Any
import jraph

Unpickling the data and importing the mesh

Create graph from mesh function

In [ ]:
def build_graph() -> jraph.GraphsTuple:
    graph = jraph
    return graph

Activation Functions

In [ ]:
def Silu(x: nnx.Array) -> nnx.Array:
    return x * nnx.sigmoid(x)

def Leaky_Relu(x: nnx.Array) -> nnx.Array:
    return nnx.leaky_relu(x)

def SoftMax(x: nnx.Array, axis: int) -> nnx.Array:
    return nnx.softmax(x, axis=axis)

Linear layer

In [ ]:
class Linear(nnx.Module):
    """
    Applies trainable linear transformation to input vector x
    Inputs: x: din dimensional row vectors as matrix
    Return: Transformed dout dimensional vector
    Trainable Params: w: d dimensional row vector, b: d dimensional row vector
    """
    def __init__(self, din: int, dout: int, rngs: nnx.Rngs):
        self.din, self.dout = din, dout
        key = rngs.params()
        initialiser = nnx.initializers.lecun_normal()
        self.w = nnx.Param(initialiser(key=key, shape=(din,dout)))
        self.b = nnx.Param(initialiser(dout,))
    
    def __call__(self, x: jnp.Array):
        return x @ self.w + self.b

GAT Layer

In [ ]:
class GAT(nnx.Module):
    def __init__(self, in_features, out_features,*,rngs):
        key = rngs.params()
        initialiser = nnx.initializers.lecun_normal()
        self.W = nnx.Param(initialiser(key=key, shape=(in_features, out_features)))
        self.a = nnx.Param(initialiser(key=key, shape=(2 * out_features, 1)))

    def __call__(self, graph: jraph.GraphsTuple) -> jraph.GraphsTuple:

        if graph.n_node is None:
            raise ValueError("GAT requires nodes to have features")
        
        tranformed_nodes = graph.nodes @ self.w

        


Model

In [ ]:
class GNN(nnx.Module):
    def __init__(self, din, embedding_dim, output_dim, rngs):
        self.embedding_layer = Linear(din, embedding_dim, rngs=rngs)
        self.decoding_layer = Linear(embedding_dim, output_dim, rngs=rngs)
    
    def embedder(self, x):
        return self.embedding_layer(x)
    
    def decoder(self, h):
        return self.decoding_layer(h)
    
    def forward_pass(self,x):
        x = self.embedder(x)
        return None
    
    def __call__():
